In [ ]:
#!/bin/bash/python

#Permutation test on length.

#180504
#originally written 180228 based on method2_v4.py script structure

#Method:
#step 1 - make 10000 random length matched shuffle bed files.
#step 2 - load each file into a for loop

In [2]:
import os, sys
import pandas
import glob
import datetime
print("start ",datetime.datetime.now())

start  2018-05-07 12:51:14.735792


In [3]:
#make a results dataframe to collect results
today = datetime.date.today()
test_path = "/dors/capra_lab/users/fongsl/roadmap/stringent/results/" # ACCRE TEMP

#mkdir = "mkdir %s "% test_path
#os.system(mkdir)
os.chdir(test_path)

In [47]:
#gather the files needed to shuffle chip-seq fragments.
blacklist = "/dors/capra_lab/users/bentonml/data/hg19_blacklist_gap.bed"
trim_chrom = "/dors/capra_lab/data/dna/hg19_trim.chrom.sizes"
v_90th = "/dors/capra_lab/users/fongsl/broadly_active_enhancers/baes_data/villar_ref_data/permutations/roadmap_vrs_0-24.bed"
v_90th = "/dors/capra_lab/users/fongsl/broadly_active_enhancers/baes_data/villar_ref_data/permutations/roadmap_vrs_25-49.bed"
v_90th = "/dors/capra_lab/users/fongsl/broadly_active_enhancers/baes_data/villar_ref_data/permutations/roadmap_vrs_50-74.bed"
v_90th = "/dors/capra_lab/users/fongsl/broadly_active_enhancers/baes_data/villar_ref_data/permutations/roadmap_vrs_75-90.bed"
v_90th = "/dors/capra_lab/users/fongsl/broadly_active_enhancers/baes_data/villar_ref_data/permutations/roadmap_vrs_90.bed"# this is a file containing the 90th% fragments from villar. 

name = (v_90th.split('/')[-1]).split('.')[0]
print(name)

roadmap_vrs_90


In [5]:
# gather all the roadmap datasets
roadmap_path = "/dors/capra_lab/users/fongsl/roadmap/stringent/" # ACCRE

roadmap_datasets= sorted(glob.glob("%sHsap_H3K27ac*.bed" %roadmap_path))

roadmap_datasets_dict= dict(((i.split("_")[-1]).split(".")[0], i) for i in roadmap_datasets) # this is a dictionary of E00*.bed:2018-01-2*.bed

In [48]:
random_results = pandas.DataFrame()
shuffle_val = 0
while shuffle_val < 3:
    random = "%s-%s.bed" % (name, shuffle_val)
    
    #shuffle the length of the fragment, matched to each chromosome. 
    shuffle_cmd = "shuffleBed -excl %s -i %s -g %s -chrom -noOverlapping > %s%s" % (blacklist, v_90th, trim_chrom, test_path, random)

    os.system(shuffle_cmd)   
    shuffle_val = shuffle_val + 1
    
    #test overlap of shuffled chromosome in each roadmap samples.
    v = pandas.read_table(random, names= ["v-chr", "v-start", "v-end"])
    v_file = test_path+random
    
    for tissue, tissue_file in roadmap_datasets_dict.items():

        outfile_u = "%s%s_u_x_%s" % (test_path, tissue, random) # make an output file name
        bed_cmd_u = "bedtools intersect -a %s -b %s -f 0.5 -e -u > %s" % (v_file, tissue_file, outfile_u)
        #print(bed_cmd_u)
        os.system(bed_cmd_u) 
        
    #gather the overlapping files, merge to find global overlap in all roadmap samples
    x = ("%s_u_x_%s") % ( "E*", random)
    any_tissue_enhancer=glob.glob(x)
    results = v.copy()
    
    for tissue in any_tissue_enhancer:

        df= pandas.read_table(tissue, names =["v-chr", "v-start", "v-end"])
        
        # name a column with the tissue ID
        tissue_id = (tissue.split("_")[0])
        
        #mark overlapping random fragments w/ 1 if random fragment appears roadmap sample
        df["%s"%tissue_id] = 1

        #left merge to count overlap of roadmap with the random_test
        results = pandas.merge(results, df, how = "left")
    
    a = results.columns[3:]
    results["sum"] = results[a].sum(axis =1)
    insert = [random.split(".")[0], results["sum"].min(), results["sum"].quantile(0.25), results["sum"].quantile(0.50), results["sum"].quantile(0.75), results["sum"].max(), results["sum"].median(), results["sum"].mean()]
    insert = pandas.Series(insert)
    random_results=random_results.append(insert, ignore_index = True)
    random_results.to_csv("%s_permutation_results.csv" % (v_90th.split('/')[9]).split(".")[0] , sep = '\t', index = False, header = True)
    print("end %s " %random ,datetime.datetime.now())

    eraser = "rm *%s*" %random 
    os.system(eraser)

end roadmap_vrs_90-0.bed  2018-05-07 14:01:13.908018
end roadmap_vrs_90-1.bed  2018-05-07 14:01:24.866789
end roadmap_vrs_90-2.bed  2018-05-07 14:01:35.816729


In [50]:
eraser = "rm *%s*" %random 
print(eraser)

rm *roadmap_vrs_90-2.bed*


In [44]:
random_results.columns =["random_itr", "min", "25th", "50th", "75th", "max", "median", "mean"]
random_results.to_csv("/dors/capra_lab/users/fongsl/broadly_active_enhancers/baes_data/villar_ref_data/permutations/%s_permutation_results.csv" % (v_90th.split('/')[9]).split(".")[0] , sep = '\t', index = False, header = True)